# Импорт всех библиотек

In [1]:
import json
import math
import os
import time
from functools import partial
from urllib.parse import urlencode

import faiss
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import requests
from catboost import CatBoostClassifier, CatBoostRanker, Pool
from faiss import read_index, write_index
from sentence_transformers import SentenceTransformer

# Подгружаем модель "ranker.ckpt"

In [2]:
model = CatBoostRanker()
model.load_model("ranker.ckpt")

# Загружаем индексы faiss

In [3]:
index = read_index("candidates.index")
index.is_trained, index.ntotal

(True, 7000000)

# Подгружаем "ind2videoid.json"

In [20]:
with open("ind2videoid.json", "r") as f:
    ind2videoid = json.load(f)

# Подгружаем тестовый датасет

In [96]:
import csv

queries = []
with open("test_dataset_submission_queries.csv", mode="r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        queries.append(row["query"])

# Загружаем модель SentenceTransformer

In [56]:
st_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

In [26]:
videos_df = pd.read_parquet(
    "videos.parquet", engine="fastparquet", columns=["video_id", "video_title"]
)

In [ ]:
%%time
def search_videos(query, topk=5):
    query_embedding = st_model.encode([query], show_progress_bar=True)[0]
    _, faiss_ind = index.search(np.array([query_embedding]), topk)

    videos_ids = [ind2videoid[f"{id}"] for id in faiss_ind[0]]

    videos_info = videos_df[videos_df["video_id"].isin(videos_ids)]

    return videos_info

with open('submission.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["query", "video"])
    for query in queries:
        videos = search_videos(query, topk=5).to_numpy()
        for video in videos:
            writer.writerow([query, video[0]])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]